In [1]:
import os
import time
import numpy as np
import pandas as pd 
from dotenv import load_dotenv

from sqlalchemy import create_engine, inspect
load_dotenv('./.env')

True

In [2]:
DB_IP = os.environ.get("DB_IP")
DB_PORT = os.environ.get("DB_PORT")
DB_NAME = os.environ.get("DB_NAME")
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")

url = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_IP}:{DB_PORT}/{DB_NAME}"
engine = create_engine(url)
inspector = inspect(engine)

In [3]:
start = time.time()
basis = pd.read_sql_table("basis_with_metrics", schema="ml_house", con=engine)
print((time.time() - start)/60)

0.9396559874216716


# posts_flags

In [4]:
start = time.time()
flags = pd.read_sql_table("posts_flags", schema="parse", con=engine)
print((time.time() - start)/60)

3.7246795376141866


In [5]:
print(flags.shape)
flags.sample()

(17544153, 13)


,id,is_post,silent,noforwards,pinned,fwd_from_flag,photo,document,web,audio,voice,video,gif
14238017,5913976,True,False,False,False,False,False,True,False,False,False,True,False


In [6]:
basis_with_flags = (
    basis
    .merge(
        flags, 
        on="id",
        how="inner"
    )
)

basis_with_flags.shape

(177373, 58)

# posts_metadata

In [8]:
start = time.time()
posts_metadata = pd.read_sql_table("posts_metadata", schema="parse", con=engine)
print((time.time() - start)/60)

6.740217089653015


In [9]:
posts_metadata = posts_metadata[["id", "channel_id", "post_date"]]

basis_with_meatadata = (
    basis_with_flags
    .merge(
        posts_metadata, 
        on="id",
        how="inner"
    )
)

basis_with_meatadata.shape

(177373, 60)

# channels

In [26]:
start = time.time()
channels = pd.read_sql_table("channels", schema="parse", con=engine)
print((time.time() - start)/60)

0.06086671749750773


In [27]:
channels

,id,name,title,participants,last_pinned_msg_id,about
0,1097289882,medicalksu,Medical Ксю,12026,5335.0,Канал о цифровых технологиях в здравоохранении...
1,1203719412,ngised,Design Porn,10048,NaN,
2,1336205252,TolmacChannel,Tolmaç Channel,445,124.0,Group of @TolmacBot\nChat @tolmacchat\nContact...
3,1742423071,kobzevii,#кобзевнасвязи,54831,9078.0,Официальный канал Губернатора Иркутской област...
4,1362285551,vlgsud,Объединенная пресс-служба судов Волгоградской ...,1659,2520.0,
...,...,...,...,...,...,...
13189,1901789788,Dekretnyeistorii,По с(д)екрету всему свету🙈,784,NaN,Привет✌🏼ты зашла в Мой маленький личный дневни...
13190,1488393404,tre_kyrsk,Тревожный Курск | Суджа,38208,6708.0,Новости Курска и Курской области.\n\nПредложит...
13191,1239513509,naufortelegram,Национальная ассоциация участников фондового р...,1871,NaN,Национальная ассоциация участников фондового р...
13192,1962245308,Kuturchin,База отдыха Кутурчин-Центр,232,742.0,


In [34]:
channels = channels[["id", "participants", "about"]]

basis_with_channels = (
    pd.merge(
        basis_with_meatadata,
        channels,
        left_on="channel_id",
        right_on="id",
        how="inner",
        suffixes=("", "_channels")
    )
)

basis_with_channels.shape

(177373, 63)

# save

In [37]:
start = time.time()
basis_with_channels.to_sql("final_basis", schema="ml_house", con=engine)
print((time.time() - start)/60)

14.239390444755553
